In [1]:
import requests
import pandas as pd
import json

import re, string, nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#key = '5oXgUp9JOCQI4zE4gEennlnQpQQzEk2OqXdq6qCW'
#key = "ai8rfuwAsAyLP2KtxWjPPCKZsthRRbIRfveJKwjY"
key = "NPIzRb1BwLXDivyCVjIdGBa6DCy0jrRB9d4AXWvj"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\burha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\burha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\burha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
def get_comments(search_term):
    # Get all dockit ids
    print("Getting Dockits of",search_term, ":....")
    url = "https://api.regulations.gov/v4/dockets?filter[searchTerm]="+search_term+"&api_key="+key
    try: dockit_ids = [docit['id'] for docit in json.loads(requests.get(url).content)['data']]
    except: 
        print("No dockit found!!!")
        return [], []
    print("Successfully got Dockits of",search_term, ":....")
    
    # Get all documents from each dockit
    print("Getting Document for each Dockit:....")
    document_object_ids = []
    for dockit in dockit_ids:
        url = "https://api.regulations.gov/v4/documents?filter[docketId]="+dockit+"&api_key="+key
        for each_doc in json.loads(requests.get(url).content)['data']: document_object_ids.append(each_doc['attributes']['objectId'])
    print("Successfully got all Document for each Dockit:....")
    
    # Get Comment ids from 
    print("Getting Comment Ids for each document:....")
    comment_ids = []
    for document in document_object_ids:
        url = "https://api.regulations.gov/v4/comments?filter[commentOnId]="+document+"&api_key="+key
        for each_doc in json.loads(requests.get(url).content)['data']: comment_ids.append(each_doc['id'])
    print("Successfully got all Comment Ids for each document:....")
    
    # Get comment from each Comment ID
    print("Getting Comment for each Comment ID:....")
    comments=[]
    for commentId in comment_ids:
        url = "https://api.regulations.gov/v4/comments/"+commentId+"?include=attachments&api_key="+key
        try: comments.append(json.loads(requests.get(url).content)['data']['attributes']['comment'])
        except: comments.append("")
    print("Got Comment for each Comment ID:....")
    
    return comment_ids, comments

In [35]:
searches = ["USA", "law", 'tax', "water", "black", "terrorism", "foreign", "nuclear", "article", "bank"]
ids, comments = [], []
for search in searches:
    print(f"-------------------- Getting Results for {search} -----------------------")
    result = get_comments(search)
    ids+=result[0]
    comments+=result[1]
    print(f"-------------------- Succcesseflly get Results for {search} -----------------------\n")

-------------------- Getting Results for USA -----------------------
Getting Dockits of USA :....
Successfully got Dockits of USA :....
Getting Document for each Dockit:....
Successfully got all Document for each Dockit:....
Getting Comment Ids for each document:....
Successfully got all Comment Ids for each document:....
Getting Comment for each Comment ID:....
Got Comment for each Comment ID:....
-------------------- Succcesseflly get Results for USA -----------------------

-------------------- Getting Results for law -----------------------
Getting Dockits of law :....
Successfully got Dockits of law :....
Getting Document for each Dockit:....
Successfully got all Document for each Dockit:....
Getting Comment Ids for each document:....
Successfully got all Comment Ids for each document:....
Getting Comment for each Comment ID:....
Got Comment for each Comment ID:....
-------------------- Succcesseflly get Results for law -----------------------

-------------------- Getting Results

In [45]:
searches = ["terrorism", "foreign", "nuclear", "article", "bank"]
for search in searches:
    print(f"-------------------- Getting Results for {search} -----------------------")
    result = get_comments(search)
    ids+=result[0]
    comments+=result[1]
    total = len(result[1])
    print(f"*********** Got {total} Comments ************\n")

-------------------- Getting Results for terrorism -----------------------
Getting Dockits of terrorism :....
Successfully got Dockits of terrorism :....
Getting Document for each Dockit:....
Successfully got all Document for each Dockit:....
Getting Comment Ids for each document:....
Successfully got all Comment Ids for each document:....
Getting Comment for each Comment ID:....
Got Comment for each Comment ID:....
*********** Got 146 Comments ************

-------------------- Getting Results for foreign -----------------------
Getting Dockits of foreign :....
Successfully got Dockits of foreign :....
Getting Document for each Dockit:....
Successfully got all Document for each Dockit:....
Getting Comment Ids for each document:....
Successfully got all Comment Ids for each document:....
Getting Comment for each Comment ID:....
Got Comment for each Comment ID:....
*********** Got 78 Comments ************

-------------------- Getting Results for nuclear -----------------------
Getting 

In [46]:
data_frame = pd.DataFrame(zip(ids, comments), columns=["CommectIds", "Comments"])
data_frame.to_csv("Comments.CSV", index=False)

In [2]:
stopwords_list = stopwords.words('english') # importing stopwords
punctuations_list = string.punctuation # get punctuations
lemmatizer = WordNetLemmatizer() # initialize word lemmatizer
def preprocessing(text):
    """
    This function will clean the givern text
    """
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text + " ".join(emoticons).replace('-', '')
    tokenize_text = [lemmatizer.lemmatize(word) for word in nltk.tokenize.word_tokenize(text) 
                     if (word not in stopwords_list) and (word not in punctuations_list) and (len(word)>=2) and (word.isalpha())]
    return " ".join(tokenize_text)

In [3]:
df = pd.read_csv("Comments.CSV")
df.dropna(inplace=True, subset=["Comments"])
comments_df = df["Comments"]

In [4]:
comments_df = comments_df.apply(preprocessing)
analyzier = SentimentIntensityAnalyzer()

In [ ]:
def sentiments(text):
    x= analyzier.polarity_scores(text)['compound']
    if x>= 0.05: return "Positive"
    elif x<= -0.05: return "Negative"
    else: return "Neutral"

def sentiments_score(text):
    return analyzier.polarity_scores(text)['compound']

In [ ]:
df["Clean Comments"] = comments_df
df['Sentiment Scores'] = comments_df.apply(sentiments_score)
df["Sentiments"] = comments_df.apply(sentiments)

In [ ]:
df.to_csv("Comments With Sentiments.CSV", index = False)